In [74]:
import hfss
from math import sin, cos, pi

In [75]:
from hfss import get_active_design
design = get_active_design()

In [76]:
a = 1.68/2
b = 8.38/2
C = 4.8*b
L = 5
d = 2.5*b
k = 10
v = 4.2*b
s = 0.5
t = 0.4
N = 60

a = design.set_variable('a', str(a)+'mm')
b = design.set_variable('b', str(b)+'mm')
C = design.set_variable('C', str(C)+'mm')
L = design.set_variable('L', str(L)+'mm')
d = design.set_variable('d', str(d)+'mm')
k = design.set_variable('k', str(k)+'mm')
v = design.set_variable('v', str(v)+'mm')
s = design.set_variable('s', str(s)+'mm')
t = design.set_variable('t', str(t)+'mm')

In [77]:
hole_location_list = [
    [0      ,       0       ,   L/2     ],
    [d      ,       0       ,   -L/2-t  ],
    [0      ,       d       ,   -L/2-t  ],
    [-d     ,       0       ,   -L/2-t  ],
    [0      ,       -d      ,   -L/2-t  ]
]
lid_location_list = [
    [0      ,       0       ,   -L/2    ],
    [d      ,       0       ,   -L/2-t-k  ],
    [0      ,       d       ,   -L/2-t-k  ],
    [-d     ,       0       ,   -L/2-t-k  ],
    [0      ,       -d      ,   -L/2-t-k  ]
]
shield_location_list = [
    [0      ,       0       ,   L/2    ],
    [d      ,       0       ,   -L/2-t-k  ],
    [0      ,       d       ,   -L/2-t-k  ],
    [-d     ,       0       ,   -L/2-t-k  ],
    [0      ,       -d      ,   -L/2-t-k  ]
]
port_location_list = [
    [0      ,       0       ,   L/2+k+t   ],
    [d      ,       0       ,   -L/2-t-k  ],
    [0      ,       d       ,   -L/2-t-k  ],
    [-d     ,       0       ,   -L/2-t-k  ],
    [0      ,       -d      ,   -L/2-t-k  ]
]
SIW1_location_list = []
for n in range(N1):
    x = v1*cos(2*n*pi/N1)
    y = v1*sin(2*n*pi/N1)
    SIW1_location_list.append([x, y, -L/2])
SIW2_location_list = []
for n in range(N2):
    x = v2*cos(2*n*pi/N2)
    y = v2*sin(2*n*pi/N2)
    SIW2_location_list.append([x, y, -L/2])

In [78]:
modeler = design.modeler
obj1 = modeler.draw_cylinder([0, 0, -L/2], C, L, axis='Z')
obj2 = modeler.draw_cylinder([0, 0, -L/2-t], C+t, L+2*t, axis='Z', name='cavity')
Cavity = modeler.subtract([obj2, obj1])
Cavity_internal = modeler.draw_cylinder([0, 0, -L/2], C, L, material='vacuum', axis='Z', name='material')

In [79]:
for loc in hole_location_list:
    obj = modeler.draw_cylinder(loc, b+t, t, axis='Z')
    Cavity = modeler.subtract([Cavity, obj])
modeler.make_perfect_E(Cavity)

In [80]:
for loc in lid_location_list:
    lead = modeler.draw_cylinder(loc, a, k+t+L, axis='Z')
    modeler.make_perfect_E(lead)
    Cavity = modeler.unite([Cavity, lead])
    obj = modeler.draw_cylinder(loc, a, k+t+L, axis='Z')
    Cavity_internal = modeler.subtract([Cavity_internal, obj])

In [81]:
for i, loc in enumerate(shield_location_list):
    obj1 = modeler.draw_cylinder(loc, b, k+t, axis='Z')
    obj2 = modeler.draw_cylinder(loc, b+t, k+t, axis='Z')
    shield = modeler.subtract([obj2, obj1])
    modeler.make_perfect_E(shield)
    Cavity = modeler.unite([Cavity, shield])
    obj = modeler.draw_cylinder(loc, a, t+k, axis='Z')
    coax_material = modeler.draw_cylinder(loc, b, k+t, material='vacuum', axis='Z')
    coax_material = modeler.subtract([coax_material, obj])
    Cavity_internal = modeler.unite([Cavity_internal, coax_material])

In [82]:
for loc in SIW_location_list:
    obj = modeler.draw_cylinder(loc, s, L, axis='Z')
    Cavity_internal = modeler.subtract([Cavity_internal, obj])
    obj = modeler.draw_cylinder(loc, s, L, axis='Z')
    modeler.make_perfect_E(obj)
    Cavity = modeler.unite([Cavity, obj])

In [13]:
port_list = []
for loc in port_location_list:
    port_list.append(modeler.draw_circle(loc, b, axis='Z'))